In [1]:
import re

file = 'data/14_input.txt'
map_length = 101
map_height = 103


# file = 'data/14_test_input.txt'
# map_length = 11
# map_height = 7

with open(file, 'r') as f:
    data = f.read().splitlines()

robots_original = {}
for id, robot in enumerate(data):
    # 'p=0,4 v=3,-3',
    # extract coordinates of p and v
    p = list(map(int, re.findall(r'-?\d+', robot)))
    robots_original[id] = ((p[0], p[1]), (p[2], p[3]))

In [2]:
def simultate_robot_movement(position, velocity):
    """
    Simuate the movement of a robot and return new position
    """

    x_move = velocity[0] + position[0]
    y_move = velocity[1] + position[1]

    if x_move < 0:
        x_move = map_length + x_move
    if y_move < 0:
        y_move = map_height + y_move

    return (x_move % map_length, y_move % map_height)

In [3]:
from copy import deepcopy

robots = deepcopy(robots_original)

for i in range(100):
    for id, pos_vel in robots.items():
        robots[id] = simultate_robot_movement(pos_vel[0], pos_vel[1]), pos_vel[1]

In [4]:
def quadrant(map_length, map_height, position):
    """
    Return the quadrant of the map for a given position
    """

    map_length = map_length - 1
    map_height = map_height - 1

    x = position[0]
    y = position[1]

    if x < map_length/2 and y < map_height/2:
        return 1
    elif x > map_length/2 and y < map_height/2:
        return 2
    elif x > map_length/2 and y > map_height/2:
        return 3
    elif x < map_length/2 and y > map_height/2:
        return 4

In [5]:
quadrants = {}

for id, pos_vel in robots.items():
    pos = pos_vel[0]
    quad = quadrant(map_length, map_height, pos)
    if quad in quadrants:
        quadrants[quad] += 1
    else:
        quadrants[quad] = 1

print(quadrants)

{4: 99, 2: 136, 1: 139, 3: 117, None: 9}


In [6]:
safety_factor = 1

for quad, count in quadrants.items():
    if quad is not None:
        safety_factor *= count

print(safety_factor)

218965032


In [7]:
import numpy, imageio

def generate_image(robots, output_file):
    """
    Generate an image of the robots
    """
    X,Y = map_length, map_height
    image = numpy.zeros((Y, X, 3), dtype=numpy.uint8)
    for _, pos_vel in robots.items():
        pos = pos_vel[0]
        image[pos[1], pos[0]] = [255, 255, 255]
    imageio.imwrite(output_file, image)

generate_image(robots)

TypeError: generate_image() missing 1 required positional argument: 'output_file'

In [12]:
def at_least_n_together_in_line(n, robots):
    """
    Check if at least `n` robots are together in a line
    """

    coords = [pos for pos, _ in robots.values()]

    for x, y in coords:
        count = 0
        for i in range(n):
            if (x+i, y) in coords:
                count += 1
            else:
                break

        if count >= n:
            return True

    return False

In [14]:
from copy import deepcopy

robots = deepcopy(robots_original)

for i in range(10000):
    for id, pos_vel in robots.items():
        robots[id] = simultate_robot_movement(pos_vel[0], pos_vel[1]), pos_vel[1]

    if at_least_n_together_in_line(7, robots):
        generate_image(robots, f'day_14_images/output{i+1}.png')

    print(f'Checked {i}', end='\r')